In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
path = '/media/data/HCPAging/data/'
folders = [i.split('/')[-1] for i in sorted(glob.glob(path+'3T_tfMRI_*'), reverse=True)]
print(folders)

['3T_tfMRI_VISMOTOR', '3T_tfMRI_FACENAME', '3T_tfMRI_CARIT']


### Converting FakeNIFTI to CIFTI files

In [ ]:
for folder in folders:
    if folder != '3T_tfMRI_CARIT':
        subjects = [i.split('/')[-1] for i in  sorted(glob.glob(path+folder+'/data/*V1_MR'))]
        for subject in subjects:
            files = sorted(glob.glob(path+folder+'/*/'+subject+'/*fakeNIFTI*feat/stats/cope*.nii.gz'))
            os.mkdir(glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti')
            for file in files:
                cmd = 'wb_command -cifti-convert -from-nifti '+ file +' '+glob.glob(path+folder+'/*/'+subject)[0]+'/template_cifti_aver.dscalar.nii'+' '+ glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti/'+file.split('/')[-1].split('.')[0]+'.unwrapped.dscalar.nii'
                os.system(cmd)
    if folder == '3T_tfMRI_CARIT':
        subjects = [i.split('/')[-1] for i in  sorted(glob.glob(path+folder+'/data/*V1_MR'))]
        for subject in subjects:
            files1 = sorted(glob.glob(path+folder+'/*/'+subject+'/*fakeNIFTI.feat/stats/cope*.nii.gz'))
            files2 = sorted(glob.glob(path+folder+'/*/'+subject+'/*fakeNIFTI+.feat/stats/cope*.nii.gz'))
            os.mkdir(glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti')
            os.mkdir(glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti_cor')
            for file in files1:
                cmd = 'wb_command -cifti-convert -from-nifti '+ file +' '+glob.glob(path+folder+'/*/'+subject)[0]+'/template_cifti_aver.dscalar.nii'+' '+ glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti/'+file.split('/')[-1].split('.')[0]+'.unwrapped.dscalar.nii'
                os.system(cmd)            
            for file in files2:
                cmd = 'wb_command -cifti-convert -from-nifti '+ file +' '+glob.glob(path+folder+'/*/'+subject)[0]+'/template_cifti_aver.dscalar.nii'+' '+ glob.glob(path+folder+'/*/'+subject)[0]+'/contrasts_cifti_cor/'+file.split('/')[-1].split('.')[0]+'.unwrapped.dscalar.nii'
                os.system(cmd)        

### Parcellate CIFTI files and extract txts

In [29]:
#atlases
parc='/media/DataD800/Alina/atlases/Q1-Q6_RelatedValidation210.CorticalAreas_dil_Final_Final_Areas_Group_Colors.32k_fs_LR.dlabel.nii'
mask_fs_path='/media/DataD800/Alina/atlases/FS_subcort'
gla_index = np.loadtxt('/media/DataD800/Alina/atlases/gla_index.txt', dtype=str)

In [51]:
for folder in folders[:-1]: #for Facename and vismot
    
    #create folder for intermediate files storing
    os.makedirs(path+folder+'/MSMAll/cort_MSMAll_av_Glasser')
    os.mkdir(path+folder+'/MSMAll/cort_MSMAll_av_Glasser_txt')
    os.mkdir(path+folder+'/MSMAll/subcort_MSMAll_con')
    os.mkdir(path+folder+'/MSMAll/subc_FS_MSMAll_txt')
    
    #read subjects IDs
    subjects = [i.split('/')[-1] for i in  sorted(glob.glob(path+folder+'/data/*V1_MR'))]
    
    #assign the number of contrast to parcellate for each task (main task contrast)
    if folder == '3T_tfMRI_FACENAME':
        con_nums = np.arange(1,7,1)
    if folder == '3T_tfMRI_VISMOTOR': 
        con_nums = [1]
    if folder == '3T_tfMRI_CARIT':
        con_nums = [1, 3, 4]
    
    for subject in subjects:
        for con_num in con_nums:
            
            #file for parcellation
            file = sorted(glob.glob(path+folder+'/*/'+subject+'/contrasts_cifti/*'+str(con_num)+'*'))[0]

            if os.path.isfile(file) == True:

                #parcellate, extract txt, separate subcortex as nii
                cmd1 = 'wb_command -cifti-parcellate ' + file +' '+ parc + ' COLUMN '+ path+folder+'/MSMAll/cort_MSMAll_av_Glasser/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.pscalar.nii' + ' -method MEAN'
                cmd2 = 'wb_command -cifti-convert -to-text '+ path+folder+'/MSMAll/cort_MSMAll_av_Glasser/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.pscalar.nii' +' '+ path+folder+'/MSMAll/cort_MSMAll_av_Glasser_txt/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.txt'
                cmd3 = 'wb_command -cifti-separate ' + file + ' COLUMN  ' + ' -volume-all ' + path+folder+'/MSMAll/subcort_MSMAll_con/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.con_subc.nii.gz'
                os.system(cmd1)
                os.system(cmd2)
                os.system(cmd3)

                #create folder for subcortex files
                os.mkdir(path+folder+'/MSMAll/subc_FS_MSMAll_txt/'+subject)

                #extract txt for each subcortex structure
                for mask in sorted(os.listdir(mask_fs_path)):
                    cmd4 = 'fslmeants -i ' + path+folder+'/MSMAll/subcort_MSMAll_con/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.con_subc.nii.gz' + ' -m ' + mask_fs_path+'/'+mask + ' -o ' + path+folder+'/MSMAll/subc_FS_MSMAll_txt/'+subject+'/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.FS_subcort.'+mask.split('.')[0]+'.mean.txt'
                    os.system(cmd4)
            else:
                print(subject, folder, ' cifti file issue, need to be reconstructed')


In [ ]:
#additional calculations for CARIT task, for correct answers only

folder='3T_tfMRI_CARIT'
#create folder for intermediate files storing
os.makedirs(path+folder+'/MSMAll_cor/cort_MSMAll_av_Glasser')
os.mkdir(path+folder+'/MSMAll_cor/cort_MSMAll_av_Glasser_txt')
os.mkdir(path+folder+'/MSMAll_cor/subcort_MSMAll_con')
os.mkdir(path+folder+'/MSMAll_cor/subc_FS_MSMAll_txt')

#read subjects IDs
subjects = [i.split('/')[-1] for i in  sorted(glob.glob(path+folder+'/data/*V1_MR'))]

#remove subjects with issue in correct answers
subjects.remove('HCA6848090_V1_MR')
subjects.remove('HCA9126269_V1_MR')


#assign the number of contrast to parcellate for each task (main task contrast)
if folder == '3T_tfMRI_FACENAME':
    con_nums = np.arange(1,7,1)
if folder == '3T_tfMRI_VISMOTOR': 
    con_nums = [1]
if folder == '3T_tfMRI_CARIT':
    con_nums = [1, 3, 4]

for subject in subjects:
    for con_num in con_nums:

        #file for parcellation
        file = sorted(glob.glob(path+folder+'/*/'+subject+'/contrasts_cifti_cor/*'+str(con_num)+'*'))[0]

        if os.path.isfile(file) == True:

            #parcellate, extract txt, separate subcortex as nii
            cmd1 = 'wb_command -cifti-parcellate ' + file +' '+ parc + ' COLUMN '+ path+folder+'/MSMAll_cor/cort_MSMAll_av_Glasser/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.pscalar.nii' + ' -method MEAN'
            cmd2 = 'wb_command -cifti-convert -to-text '+ path+folder+'/MSMAll_cor/cort_MSMAll_av_Glasser/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.pscalar.nii' +' '+ path+folder+'/MSMAll_cor/cort_MSMAll_av_Glasser_txt/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.aver_parc.txt'
            cmd3 = 'wb_command -cifti-separate ' + file + ' COLUMN  ' + ' -volume-all ' + path+folder+'/MSMAll_cor/subcort_MSMAll_con/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.con_subc.nii.gz'
            os.system(cmd1)
            os.system(cmd2)
            os.system(cmd3)

            #create folder for subcortex files
            os.mkdir(path+folder+'/MSMAll_cor/subc_FS_MSMAll_txt/'+subject)

            #extract txt for each subcortex structure
            for mask in sorted(os.listdir(mask_fs_path)):
                cmd4 = 'fslmeants -i ' + path+folder+'/MSMAll_cor/subcort_MSMAll_con/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.MSMAll.con_subc.nii.gz' + ' -m ' + mask_fs_path+'/'+mask + ' -o ' + path+folder+'/MSMAll_cor/subc_FS_MSMAll_txt/'+subject+'/'+subject+'.'+file.split('/')[-1].split('.')[0]+'.FS_subcort.'+mask.split('.')[0]+'.mean.txt'
                os.system(cmd4)
        else:
            print(subject, folder, ' cifti file issue, need to be reconstructed')

##### Assembling scv tables 

In [103]:
for folder in folders:

    subjects = [i.split('/')[-1].split('.')[0] for i in sorted(glob.glob(path+folder+'/MSMAll/cort_MSMAll_av_Glasser_txt/*'))]

    dct_group = {}
    for subject in subjects:
        #reading file with Glasser atlas parcellation
        df_cort = pd.read_csv(glob.glob(path+folder+'/MSMAll/cort_MSMAll_av_Glasser_txt/'+subject+'*')[0], 
                              header=None, squeeze=True)
        df_cort.index = gla_index

        #reading subcortex files
        dct_subc = {}
        for file in sorted(glob.glob(path+folder+'/MSMAll/subc_FS_MSMAll_txt/'+subject+'/*')):
            dct_subc[str(file.split('.')[-3])] = np.loadtxt(file)
        df_subc = (pd.Series(dct_subc))

        dct_group[subject] = pd.concat([df_cort,df_subc], axis=0)

    #save table
    df_group = pd.DataFrame(dct_group).T
    df_group.to_csv(path+folder+'/'+folder.split('_')[-1]+'_group_table.csv')